In [1]:
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display

df_filtered_global = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\jupyter\OEM\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global
    df_filtered_global = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DTCONCLUSAO"] >= data_inicial) &
        (df["DTCONCLUSAO"] <= data_final)
    ]
    return df_filtered_global

if __name__ == "__main__":
    df = ler_planilha()

    df["DTCONCLUSAO"] = pd.to_datetime(df["DTCONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = list(df['STATUS'].unique())
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = list(df['TIPO'].unique())
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = list(df['ORIGEM'].unique())
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = list(df['TIPOCAUSA'].unique())
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DTCONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DTCONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)

    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(1900, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 2), step=1)

In [23]:
display(df_filtered_global)

,Unnamed: 0,ORIGEM,TIPO,NUMOS,NUMOCORRENCIA,ROTA,NOMECLIENTE,CPF,CODIGOENERGISA,UC,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG
2026,False,ENERGISA,PREVENTIVA,202400002954,NaN,4.0,FRANCISCA DAS CHAGAS VALERIANO DA SILVA,665.300.912-20,MLPA6709948-21,658004.0,...,34,2024-01-17 14:57:00,2024-01-17 15:06:00,2024-01-09 08:31:00,3,2024-01-17,202400002954,Report2,2666,NaN


In [20]:
df_filtered_global.loc[:, ['N/C', 'ATUALIZACAO_2', 'DTLOG']] = np.nan
# Desmenbrando o Gps LATLONCON.
def split_lat_lon(x):
    if isinstance(x, str):
        return pd.Series(x.split(','))
    else:
        return pd.Series([None, None])

df_filtered_global[['LATITUDE', 'LONGITUDE','erro_1','erro_2']] = df_filtered_global['LATLONCON'].apply(split_lat_lon)#'erro_1','erro_2'

C:\Users\helym\AppData\Local\Temp\ipykernel_1028\2267655168.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_global[['LATITUDE', 'LONGITUDE','erro_1','erro_2']] = df_filtered_global['LATLONCON'].apply(split_lat_lon)#'erro_1','erro_2'
C:\Users\helym\AppData\Local\Temp\ipykernel_1028\2267655168.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_global[['LATITUDE', 'LONGITUDE','erro_1','erro_2']] = df_filtered_global['LATLONCON'].apply(split_lat_lon)#'erro_1','erro_2'
C:\Users\helym

In [21]:
df_conclusao =df_filtered_global[['DTCONCLUSAO','EXECUTOR','LATITUDE','LONGITUDE','ORIGEM', 'TIPO', 'NUMOS','IDSIGFI','NUMOCORRENCIA', 'ROTA',
       'NOMECLIENTE', 'CPF','STATUS']]
dados_analise = df_filtered_global

In [22]:
# Criar a tabela dinâmica com a contagem total
tabela_dinamica = pd.pivot_table(df_filtered_global,
                                  values=['NUMOS'],  # Coluna para resumir
                                  index=['STATUS'],  # Índice da tabela dinâmica
                                  columns=['ROTA'],  # Colunas da tabela dinâmica
                                  aggfunc={'NUMOS': 'count'},  # Função de agregação
                                  fill_value=0,  # Valor de preenchimento para células vazias
                                  margins=True,  # Adiciona estatísticas totais
                                  margins_name='Total')  # Nome da linha e coluna de totais

# Agora, crie a tabela dinâmica a partir do DataFrame filtrado
tabela_dinamica_1 = pd.pivot_table(df_filtered_global,
                                  values=['NUMOS'],  # Coluna para resumir
                                  index=['EXECUTOR'],  # Índice da tabela dinâmica
                                  columns=['ROTA'],  # Colunas da tabela dinâmica
                                  aggfunc={'NUMOS': 'count'},  # Função de agregação
                                  fill_value=0)  # Valor de preenchimento para células vazias
                                
tabela_dinamica_2 = pd.pivot_table(df_filtered_global,
                                   values='NUMOS',  # Coluna para resumir
                                   index='EXECUTOR',  # Índice da tabela dinâmica
                                   columns='DTCONCLUSAO',  # Colunas da tabela dinâmica
                                   aggfunc= 'count',  # Função de agregação para contar
                                   fill_value=0,  # Valor de preenchimento para células vazias
                                   margins=True,  # Adiciona estatísticas totais
                                   margins_name='Total')  # Nome da linha e coluna de totais
                            
                                
# Agora, crie a tabela dinâmica a partir do DataFrame filtrado
tabela_dinamica_3 = pd.pivot_table(df_filtered_global,
                                  values=['NUMOS'],  # Coluna para resumir
                                  index=['ROTA'],  # Índice da tabela dinâmica
                                  columns=['TIPOCAUSA'],  # Colunas da tabela dinâmica
                                  aggfunc={'TIPOCAUSA': 'count'},  # Função de agregação
                                  fill_value=0)  # Valor de preenchimento para células vazias

print('******************************************************************************************************************************************')
display(tabela_dinamica_2)
print('******************************************************************************************************************************************')
display(tabela_dinamica)
print('******************************************************************************************************************************************')
display(tabela_dinamica_1)
print('******************************************************************************************************************************************')
display(tabela_dinamica_3)

******************************************************************************************************************************************


DTCONCLUSAO,2022-09-17,2022-09-18,2022-09-20,2022-09-21,2022-09-22,2022-09-23,2022-09-24,2022-09-25,2022-09-27,2022-09-28,...,2024-04-24,2024-04-25,2024-04-26,2024-04-27,2024-04-28,2024-04-29,2024-04-30,2024-05-01,2024-05-02,Total
EXECUTOR,,,,,,,,,,,,,,,,,,,,,
EDUARDO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,56
MARNEI,0,0,0,7,0,0,0,1,1,2,...,0,0,0,0,0,0,0,0,0,22
ONEL004,0,0,0,0,0,0,0,0,0,0,...,1,0,0,3,0,0,0,0,0,203
ONEL007,0,0,0,0,0,0,0,0,0,0,...,4,4,3,3,3,3,3,0,0,229
ONEL012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,231
ONEOM001,1,1,3,1,2,2,2,0,3,2,...,0,0,0,0,0,0,0,0,0,102
ONEOM002,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,579
ONEOM003,0,0,0,0,1,0,0,0,0,2,...,2,4,1,2,0,1,0,0,0,161
ONEOM004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,276


******************************************************************************************************************************************


NUMOS                                                ...       \
ROTA           1.0  2.0   3.0  4.0  5.0  6.0  7.0  8.0  9.0 10.0  ... 12.0   
STATUS                                                            ...        
EM ANALISE       0    0     0    0    0    0    0    4    0    0  ...    0   
ENVIADO        187   34   373   29   84   84  155  150   62   49  ...    0   
NAO CONFORME     0    0     0    0    0    0    2    0    0    0  ...    0   
VALIDADO       626   96  1031   96  247  252  443  410  199  206  ...   90   
Total          813  130  1404  125  331  336  600  564  261  255  ...   90   

                                                            
ROTA         13.0 14.0 15.0 16.0 21.0 22.0 53.0 54.0 Total  
STATUS                                                      
EM ANALISE      0    0    0    0    0    0    0    0     4  
ENVIADO         0    0    0    0    0    0    0    0  1208  
NAO CONFORME    0    0    0    0    6    0    0    0     8  
VALIDADO       48   37    6  181  239   46   48   83  4551  
Total          48   37    6  181  245   46   48   83  5771  

[5 rows x 21 columns]

******************************************************************************************************************************************


NUMOS                                                              \
ROTA      1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0 11.0 12.0 13.0   
EXECUTOR                                                                     
EDUARDO      0    0    0    0    0    0    0    0    0    0   56    0    0   
MARNEI      16    0    6    0    0    0    0    0    0    0    0    0    0   
ONEL004      0    0    0    0    0   55    0    0    0    0    0    0    0   
ONEL007      0    0    0    0    0    0    0    0    0    0    0    0    0   
ONEL012      0    0  231    0    0    0    0    0    0    0    0    0    0   
ONEOM001     3    0    0    0   83    0    0    0    0    0    0    0   16   
ONEOM002   316    0    0   31    0   83    0  149    0    0    0    0    0   
ONEOM003    45    0    2    0    0    0    0    0    0    0    1   30    0   
ONEOM004     0    0    0    0    0   29  147    0    0   85    1   14    0   
ONEOM005    47    0  110   42    0   85   98  149   86    0    0    0    0   
ONEOM006     0    0    5    0    0    0    0   80    0    0    0    0    0   
ONEOM007     1    0  160   27  248    0  301   20    0  127   55    0   16   
ONEOM008    92    0    0    0    0    0    0   97    0    0    0    0    0   
ONEOM009     0   33  173    0    0   84    2    0  175   43    0    0   16   
ONEOM011     0    0   92   25    0    0    0   69    0    0   55   16    0   
ONEOM012   131   33  352    0    0    0    0    0    0    0    0   30    0   
ONEOM013   104   64  190    0    0    0   47    0    0    0    0    0    0   
ONEOM014     0    0   48    0    0    0    5    0    0    0    0    0    0   
ONEOM015    58    0    0    0    0    0    0    0    0    0    0    0    0   
ROTA12B      0    0   35    0    0    0    0    0    0    0    0    0    0   

                                             
ROTA     14.0 15.0 16.0 21.0 22.0 53.0 54.0  
EXECUTOR                                     
EDUARDO     0    0    0    0    0    0    0  
MARNEI      0    0    0    0    0    0    0  
ONEL004     0    0    0  148    0    0    0  
ONEL007     0    0  181    0    0   48    0  
ONEL012     0    0    0    0    0    0    0  
ONEOM001    0    0    0    0    0    0    0  
ONEOM002    0    0    0    0    0    0    0  
ONEOM003    0    0    0    0    0    0   83  
ONEOM004    0    0    0    0    0    0    0  
ONEOM005    0    0    0    0    0    0    0  
ONEOM006   37    6    0   60   46    0    0  
ONEOM007    0    0    0   37    0    0    0  
ONEOM008    0    0    0    0    0    0    0  
ONEOM009    0    0    0    0    0    0    0  
ONEOM011    0    0    0    0    0    0    0  
ONEOM012    0    0    0    0    0    0    0  
ONEOM013    0    0    0    0    0    0    0  
ONEOM014    0    0    0    0    0    0    0  
ONEOM015    0    0    0    0    0    0    0  
ROTA12B     0    0    0    0    0    0    0

******************************************************************************************************************************************


TIPOCAUSA                                          \
TIPOCAUSA INTERMITENCIA MANUT PREVENTIVA 1A MANUT PREVENTIVA 2A   
ROTA                                                              
1.0                   0                 205                 204   
2.0                   0                  33                  33   
3.0                   0                 362                 362   
4.0                   1                  42                  42   
5.0                   0                  84                  83   
6.0                   0                  84                  84   
7.0                   0                 151                 150   
8.0                   0                 149                 149   
9.0                   0                  86                  87   
10.0                  0                  85                  85   
11.0                  0                  56                  57   
12.0                  0                  30                  30   
13.0                  0                  16                  16   
14.0                  0                  37                   0   
15.0                  0                   6                   0   
16.0                  0                 181                   0   
21.0                  0                 245                   0   
22.0                  0                  46                   0   
53.0                  0                  48                   0   
54.0                  0                  83                   0   

                                                   
TIPOCAUSA MANUT PREVENTIVA 3A MANUT PREVENTIVA 4A  
ROTA                                               
1.0                       204                 200  
2.0                        33                  31  
3.0                       360                 320  
4.0                        40                   0  
5.0                        82                  82  
6.0                        84                  84  
7.0                       151                 148  
8.0                       149                 117  
9.0                        88                   0  
10.0                       85                   0  
11.0                       55                   0  
12.0                       30                   0  
13.0                       16                   0  
14.0                        0                   0  
15.0                        0                   0  
16.0                        0                   0  
21.0                        0                   0  
22.0                        0                   0  
53.0                        0                   0  
54.0                        0                   0

In [16]:
validation_preventiva = df_filtered_global[df_filtered_global['TIPO'] == 'PREVENTIVA']
validation_corretiva = df_filtered_global[df_filtered_global['TIPO'] == 'CORRETIVA']

preventive = validation_preventiva[[
    'N/C', 'ATUALIZACAO_2', 'DTLOG', 'STATUS', 'EXECUTOR', 'DTCONCLUSAO', 'ORIGEM', 'TIPO', 'NUMOS', 'ROTA',
    'NOMECLIENTE', 'CPF', 'CODIGOENERGISA', 'UC', 'IDSIGFI', 'QDCACCNS',
    'OBRA', 'TIPODESIGFI', 'DTAINC', 'DTAALT', 'USUARIOINC',
    'USUARIOALT', 'LATLON', 'LATLONCON', 'NOMEDAMAE',
    'RECLAMANTE', 'MUNICIPIO', 'ENDERECO', 'COMPETENCIA', 'CAUSA',
    'TIPOCAUSA', 'DTAHORARECLAMACAO', 'TELEFONES', 'CONCLUSAO',
    'DIAGNOSTICO', 'DESCADICIONALPROG', 'DESCADICIONALEXEC',
    'IDTIPOMANUTENCAO', 'IDORIGEM', 'IDTIPOSOLICITACAO', 'IDCAUSA',
    'IDCONCLUSAO', 'IDEXECUTOR', 'IDDIAGNOSTICO', 'DTINIDESLOCAMENTO',
    'DTCHEGADA', 'DTAHORAFATO', 'STATUSSRV', 'DATACONCLUSAO', 'IDMOBILE',
    'MANUTENCAO'
]]

corrective = validation_corretiva[[
    'N/C', 'ATUALIZACAO_2', 'DTLOG', 'STATUS', 'EXECUTOR', 'DTCONCLUSAO', 'ORIGEM', 'TIPO', 'NUMOS', 'NUMOCORRENCIA', 'ROTA',
    'NOMECLIENTE', 'CPF', 'CODIGOENERGISA', 'UC', 'IDSIGFI', 'QDCACCNS',
    'OBRA', 'TIPODESIGFI', 'DTAINC', 'DTAALT', 'USUARIOINC',
    'USUARIOALT', 'LATLON', 'LATLONCON', 'NOMEDAMAE',
    'RECLAMANTE', 'MUNICIPIO', 'ENDERECO', 'COMPETENCIA', 'CAUSA',
    'TIPOCAUSA', 'DTAHORARECLAMACAO', 'TELEFONES', 'CONCLUSAO',
    'DIAGNOSTICO', 'DESCADICIONALPROG', 'DESCADICIONALEXEC',
    'IDTIPOMANUTENCAO', 'IDORIGEM', 'IDTIPOSOLICITACAO', 'IDCAUSA',
    'IDCONCLUSAO', 'IDEXECUTOR', 'IDDIAGNOSTICO', 'DTINIDESLOCAMENTO',
    'DTCHEGADA', 'DTAHORAFATO', 'STATUSSRV', 'DATACONCLUSAO', 'IDMOBILE',
    'MANUTENCAO'
]]

nc_os = validation_preventiva[['NUMOS','ROTA','UC','IDSIGFI','EXECUTOR','NOMECLIENTE','DESCADICIONALPROG','STATUS','DTCONCLUSAO','LATITUDE', 'LONGITUDE']]

In [17]:
prod_preventiva = validation_preventiva[['NUMOS','TIPOCAUSA','UC','IDSIGFI','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]
prod_corretiva = validation_corretiva[['NUMOS','NUMOCORRENCIA','TIPOCAUSA','UC','IDSIGFI','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

In [18]:
# Cria planilha guia para o validador se orientar
with pd.ExcelWriter(r'C:\Users\helym\projetos_python\jupyter\OEM\output\validacao_o&m_interbras_09_05_2024.xlsx', engine='openpyxl') as writer:
    # Salva os dados do dataframe 'validation' na primeira aba
    preventive.to_excel(writer, sheet_name='preventiva_energisa', index=False)

    # Salva os dados do dataframe 'prod_2' na segunda aba
    corrective.to_excel(writer, sheet_name='corretiva_energisa', index=False)

In [ ]:
# Salvando a planilha( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\OEM\output\base_sip_oem1tranche_preventiva_geral.xlsx")#"E:\OEM\output\base_sip_oem_preventiva_geral.xlsx"
preventive.to_excel(dataframe_excel, index=False)
dataframe_excel.close()

In [ ]:
# Salvando a planilha( .xlsx)
dataframe_excel = pd.ExcelWriter(r'C:\Users\helym\projetos_python\jupyter\OEM\output\nc_oem_geral.xlsx')
df_filtered_global.to_excel(dataframe_excel, index=False)
dataframe_excel.close()

In [15]:
producao = dados_analise.copy()
quantidade = dados_analise['EXECUTOR'].count()
agrupamento_tipo = dados_analise.groupby('TIPO').count()
tipo_preventiva = sum(1 for tipo in dados_analise['TIPO']if 'PREVENTIVA' in tipo)
tipo_corrretiva = sum(1 for tipo in dados_analise['TIPO']if 'CORRETIVA' in tipo)
status_valido = sum(1 for tipo in dados_analise['STATUS']if 'VALIDADO'in tipo or'ENVIADO' in tipo)
status_nc = sum(1 for tipo in dados_analise['STATUS']if 'NAO CONFORME' in tipo)
rota = ', '.join(str(valor) for valor in dados_analise["ROTA"].unique())
tipo_manut = agrupamento_tipo[['NUMOS']]
data_selecionada = selected_date.strftime('%d-%m-%Y')
#data_selecionada = selected_date
equipe = selected_executor
dtlog_ausente = input("Quantos datalogs estão ausente?")
observacao = input("Existe alguma observação a ser pontuada?")

NameError: name 'selected_date' is not defined

In [ ]:
espaco = "             "
mensagem = (
    f"Usuário:{equipe}{espaco}Rota:{rota}\n"
    f"Data da Produção:          {data_selecionada}\n\n"
    f"Preventiva executada: {tipo_preventiva}\n"
    f"Corretiva executada: {tipo_corrretiva}\n"
    f"Total Executado: {quantidade}\n"
    f"Validados: {status_valido}\n"
    f"Não Conforme: {status_nc}\n"
    f"DTLOG ausentes: {dtlog_ausente}\n"
    f"Observação:  {observacao}"
)

print(mensagem)